In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Configuration
BATCH_SIZE=32

In [3]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download=True,
    transform=ToTensor(),
)


test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of X[N,C,H,W]: {X.shape}")
    print(f"Sahep of y:{y.shape} {y.dtype} ")
    break

Shape of X[N,C,H,W]: torch.Size([32, 1, 28, 28])
Sahep of y:torch.Size([32]) torch.int64 


In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.op1 = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            
        )
        self.opt2 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256,10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.op1(x)
        logits = self.opt2(x)
        
        return logits
    
model = NeuralNetwork().to(device)
print(model)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (op1): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
  )
  (opt2): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}|{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296831 [   32|60000]
loss: 1.063768 [ 3232|60000]
loss: 0.529172 [ 6432|60000]
loss: 0.714835 [ 9632|60000]
loss: 0.505700 [12832|60000]
loss: 0.879937 [16032|60000]
loss: 0.401165 [19232|60000]
loss: 0.349049 [22432|60000]
loss: 0.423162 [25632|60000]
loss: 0.344519 [28832|60000]
loss: 0.589048 [32032|60000]
loss: 0.629595 [35232|60000]
loss: 0.446327 [38432|60000]
loss: 0.737089 [41632|60000]
loss: 0.650426 [44832|60000]
loss: 0.396568 [48032|60000]
loss: 0.476583 [51232|60000]
loss: 0.513537 [54432|60000]
loss: 0.615063 [57632|60000]
Test Error: 
 Accuracy: 83.4%, Avg loss: 0.458544 

Epoch 2
-------------------------------
loss: 0.511790 [   32|60000]
loss: 0.397935 [ 3232|60000]
loss: 0.236208 [ 6432|60000]
loss: 0.481069 [ 9632|60000]
loss: 0.330242 [12832|60000]
loss: 0.615726 [16032|60000]
loss: 0.361917 [19232|60000]
loss: 0.351749 [22432|60000]
loss: 0.423872 [25632|60000]
loss: 0.255509 [28832|60000]
loss: 0.492824 [32032|60000

loss: 0.609520 [48032|60000]
loss: 0.155000 [51232|60000]
loss: 0.278620 [54432|60000]
loss: 0.274683 [57632|60000]
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.369731 

Epoch 14
-------------------------------
loss: 0.152976 [   32|60000]
loss: 0.265663 [ 3232|60000]
loss: 0.186386 [ 6432|60000]
loss: 0.259214 [ 9632|60000]
loss: 0.216184 [12832|60000]
loss: 0.477608 [16032|60000]
loss: 0.152035 [19232|60000]
loss: 0.114043 [22432|60000]
loss: 0.230078 [25632|60000]
loss: 0.153460 [28832|60000]
loss: 0.419055 [32032|60000]
loss: 0.464533 [35232|60000]
loss: 0.313116 [38432|60000]
loss: 0.395047 [41632|60000]
loss: 0.459386 [44832|60000]
loss: 0.404704 [48032|60000]
loss: 0.149362 [51232|60000]
loss: 0.563296 [54432|60000]
loss: 0.303214 [57632|60000]
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.374183 

Epoch 15
-------------------------------
loss: 0.158540 [   32|60000]
loss: 0.263657 [ 3232|60000]
loss: 0.152882 [ 6432|60000]
loss: 0.371064 [ 9632|60000]
loss: 0.356608 [12832|60000]
lo

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
